In [1]:
%%time 
from data.pair_feature import *
from data.original import item_pairs_train, item_pairs_test, item_info_train, item_info_test
from data.corpus_based import feature_nodes as corpus_based_feature_nodes
# from data.dummy import dummy_features
from data.item import get_item, show_item_pair
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
n_train = 3344613
n_test = 1315205

CPU times: user 1min, sys: 14.9 s, total: 1min 15s
Wall time: 1min 15s


In [2]:
%%time
image_features_train = pd.read_csv('data/data_files/image_feature_train.csv')
image_features_test = pd.read_csv('data/data_files/image_feature_test.csv')

CPU times: user 7.52 s, sys: 316 ms, total: 7.83 s
Wall time: 7.84 s


In [6]:
print(image_features_train.shape)
print(image_features_test.shape)

(2991396, 9)
(1044196, 9)


In [7]:
corpus_based_features = []
for node in corpus_based_feature_nodes:
    corpus_based_features.append(node.get_data())
corpus_based_features = pd.concat(corpus_based_features, axis=1)

In [8]:
corpus_based_features.shape

(4035592, 96)

In [9]:
def split_feats(feats):
    n_train = 2991396
    feats_train = feats[:n_train]
    feats_test = feats[n_train:]
    feats_test.index = simple_features_test.index.copy()
    return feats_train, feats_test

In [10]:
corpus_based_features_train, corpus_based_features_test = split_feats(corpus_based_features)
# dummy_features_train, dummy_features_test = split_feats(dummy_features)

In [17]:
features_train = pd.concat((simple_features_train, aggregation_features_train, title_features_train, description_features_train,
#                             dummy_features_train,
                            image_features_train,
                            corpus_based_features_train), axis=1)
features_test = pd.concat((simple_features_test, aggregation_features_test, title_features_test, description_features_test,
#                            dummy_features_test,
                           image_features_test, 
                           corpus_based_features_test), axis=1)

In [18]:
print(features_train.shape)
print(features_test.shape)

(2991396, 216)
(1044196, 216)


In [19]:
%%time
X_train, X_test, y_train, y_test, I_train, I_test =\
train_test_split(features_train.values, item_pairs_train.isDuplicate.values, np.arange(features_train.shape[0]),
                 test_size=0.2, random_state=42)

CPU times: user 3min 15s, sys: 23.4 s, total: 3min 38s
Wall time: 3min 38s


In [20]:
item_info_train.categoryID.unique().shape

(51,)

In [21]:
model = XGBClassifier(learning_rate=0.1, max_depth=10, subsample=0.8, colsample_bytree=0.8, n_estimators=260,
                     nthread=32)

In [ ]:
model.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_test, y_test)], early_stopping_rounds=20)

In [23]:
? model.fit

In [24]:
%time pred_test = model.predict_proba(X_test)[:,1]

CPU times: user 1min 16s, sys: 1.74 s, total: 1min 18s
Wall time: 34.5 s


In [25]:
eval_set = item_pairs_train.iloc[I_test]
eval_set['prob_predicted'] = pred_test

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [26]:
from bokeh.io import output_notebook, show
from bokeh.charts import Histogram
output_notebook()

Loading BokehJS ...

In [27]:
from ml_metrics import auc

In [28]:
eval_set.groupby('isDuplicate')['prob_predicted'].mean()

isDuplicate
0    0.145486
1    0.799383
Name: prob_predicted, dtype: float32

In [29]:
eval_set.groupby('isDuplicate')['prob_predicted'].std()

isDuplicate
0    0.206541
1    0.253222
Name: prob_predicted, dtype: float32

In [30]:
eval_set['categoryID'] = eval_set.itemID_1.apply(lambda x: get_item(x)['categoryID'])

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
%time eval_set['categoryID'] = eval_set.itemID_1.apply(lambda x: get_item(x)['categoryID'])

CPU times: user 9.76 s, sys: 12 ms, total: 9.78 s
Wall time: 9.86 s


/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
auc(eval_set.isDuplicate.values, eval_set.prob_predicted.values)

0.9560049309562599

In [33]:
for i in [1,3]:
    tmp = eval_set[eval_set.generationMethod==i]
    print('generation method {} auc: {}'.format(i, auc(tmp.isDuplicate.values, tmp.prob_predicted.values)))

generation method 1 auc: 0.9388163450266627
generation method 3 auc: 0.9781376246263175


In [34]:
auc_by_category = eval_set.groupby('categoryID')[['isDuplicate', 'prob_predicted']].apply(lambda x: auc(x['isDuplicate'].values, x['prob_predicted'].values))

In [35]:
auc_by_category.order()

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


categoryID
112    0.853727
85     0.890865
33     0.908268
105    0.918660
101    0.918990
99     0.923546
111    0.927191
10     0.927878
31     0.929550
34     0.936885
19     0.938216
97     0.939046
87     0.941084
39     0.943741
23     0.944613
21     0.945017
32     0.945380
40     0.946216
98     0.946767
25     0.946779
26     0.947870
29     0.949174
115    0.950093
27     0.950440
86     0.950804
36     0.952383
42     0.952914
94     0.954811
84     0.955453
24     0.955598
83     0.958183
96     0.959649
38     0.960350
102    0.960493
90     0.960758
28     0.961039
20     0.962766
89     0.969147
114    0.971424
30     0.971867
88     0.972333
9      0.975524
106    0.976134
93     0.978428
82     0.979361
14     0.980515
81     0.980925
92     0.983258
91     0.985710
11     0.987732
116    0.990506
dtype: float64

In [36]:
from bokeh.charts import Bar

In [37]:
show(Bar(auc_by_category.order()))

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


In [38]:
eval_set.shape[0]

598280

In [39]:
p = Histogram(eval_set.head(5000), values='prob_predicted', color='isDuplicate', bins=30, legend=True, plot_width=1000)
show(p)

In [40]:
tmp = eval_set[(eval_set.prob_predicted>=0.4) & (eval_set.prob_predicted<=0.6)].sort(['isDuplicate', 'prob_predicted'], ascending=False)

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [41]:
tmp.head()

,itemID_1,itemID_2,isDuplicate,generationMethod,prob_predicted,categoryID
2964546,5530441,5792940,1,1,0.599999,96
2221642,2995274,4436984,1,1,0.599997,20
2410732,3405871,5607835,1,1,0.599990,89
1554656,1862380,2759595,1,1,0.599989,19
2759120,4398548,6079281,1,1,0.599987,9


In [42]:
show_item_pair(5029691, 5390935)

In [43]:
features_train.loc[2898895]

attrsJSON_item_jaccard                                    1
attrsJSON_key_jaccard                                     1
description_length_diff                                   0
description_length_diff_ratio                             0
description_length_max                                  324
description_length_min                                  324
images_count_diff                                         0
images_count_diff_ratio                                   0
images_count_max                                          1
images_count_min                                          1
location_distance                                         0
price_diff                                                0
price_diff_ratio                                          0
price_max                                             17600
price_min                                             17600
same_categoryID                                        True
same_description                        

In [44]:
feature_importance = pd.DataFrame({'feature': features_train.columns,
              'importance': model.feature_importances_}).sort('importance', ascending=False)

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [45]:
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr

In [46]:
feature_importance.head(50)

,feature,importance
4,description_length_max,0.022113
5,description_length_min,0.019147
14,price_min,0.016887
13,price_max,0.016327
11,price_diff,0.016186
112,max_hist_diff_8bins,0.015788
43,title_2_price_mean,0.014998
26,categoryID_freq,0.014628
117,min_hist_diff_64bins,0.014594
39,title_1_price_mean,0.014011


In [47]:
feature_importance.set_index('feature').loc[corpus_based_features.columns].sort('importance', ascending=False)

/home/idm/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,importance
description_word_dtm_0__max_disjoint_idf_min,0.011864
description_word_dtm_2__max_disjoint_idf_min,0.008976
title_word_dtm_0__max_disjoint_idf_min,0.008937
title_word_lsa_1_0__tf,0.006924
description_word_dtm_2__max_disjoint_idf_max,0.006874
description_word_dtm_0__max_disjoint_idf_max,0.006840
title_word_lsa_1_1__tf,0.006812
description_word_lsa_1_0__tf,0.006571
description_word_lsa_1_2__tf,0.006425
description_word_lsa_1_3__tf,0.006369


In [48]:
show(Bar(feature_importance.head(50), label=CatAttr(columns=['feature'], sort=False), values='importance',
        plot_width=1000))

In [49]:
%%time
model.fit(features_train, item_pairs_train.isDuplicate)
prob_test = model.predict_proba(features_test)[:,1]
pd.DataFrame({'id': item_pairs_test['id'], 'probability': prob_test}).to_csv('baseline.17.csv', index=False)

CPU times: user 17h 57min 34s, sys: 8min 39s, total: 18h 6min 14s
Wall time: 36min 14s
